In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from models.VAE import VanillaVAE, IWAE, VectorQuantizedVAE

### Toy example: 8 gaussians

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def eightgaussian(n_points):
    """
     Returns the eight gaussian dataset.
    """
    n = np.random.randint(0,8, n_points)
    noisex = np.random.normal(size=(n_points)) * 0.2
    noisey = np.random.normal(size=(n_points)) * 0.2
    x_centers,y_centers = [np.cos(n* np.pi/4.0) * 5 + noisex, np.sin(n* np.pi/4.0) * 5 + noisey]
    return np.vstack((x_centers,y_centers)).T
            
X = eightgaussian(10000)
X_test = eightgaussian(5000)
X.shape
plt.figure(figsize=(6,6))
plt.scatter(X[:,0], X[:,1], s=1);
plt.axis('square');

In [ ]:
from utils.data import make_dataloaders, save_model, DotDict
import torch
from tqdm import tqdm
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
train_loader = torch.utils.data.DataLoader(X, batch_size=32, collate_fn=lambda x:torch.Tensor(x))
val_loader = torch.utils.data.DataLoader(X_test, batch_size=32, collate_fn=lambda x:torch.Tensor(x))

clipping_value=0.1

def train(model, epochs):
    losses = []
    val_losses = []
    model.to(device)
    optim = model.get_optimizer()

    for epoch in tqdm(range(epochs)):
        for batch in train_loader:
            optim.zero_grad()
            loss, x_hat, z, BCE = model.step(batch)
            losses.append(loss.item())
            loss.backward()
            optim.step()
        
    print(f"{model.description} epoch: {epoch} loss:{losses[-1]:.2}")


In [ ]:
def display_vq(model, dims=[0,1]):
    
    x_tilde, z_e_x, z_q_x = model.forward(torch.Tensor(X_test[:300]))
    rec_x = x_tilde.detach().cpu().numpy()
    z_e_x=z_e_x.detach().cpu().numpy()
    z_q_x=z_q_x.detach().cpu().numpy()
    fig = plt.figure(figsize=(10, 10))
    plt.scatter(X_test[:300, 0], X_test[:300, 1], s=2, c="y", alpha=0.5, label="x")
    plt.scatter(z_e_x[:, dims[0]], z_e_x[:, dims[1]], s=2, c="g", alpha=0.5, label="z_e_x")
    plt.scatter(z_q_x[:, dims[0]], z_q_x[:, dims[1]], s=100, edgecolors="b", label="z_q_x", alpha=1, marker="o", facecolors='none')
    plt.scatter(rec_x[:, 0], rec_x[:, 1], s=100, edgecolors="r", alpha=1.0,label="reconstructed", marker="o", facecolors='none')
    ax = fig.axes[0]
    style = dict(size=20, color='gray')
    embs = model.codebook.embedding.weight.data
    embs_dec = model.net.decode(embs).data
    plt.scatter(embs[:, dims[0]], embs[:, dims[1]], s=20, c="b", label="embeddings", marker="x")
    plt.scatter(embs_dec[:, dims[0]], embs_dec[:, dims[1]], s=20, c="r", label="embeddings_dec", marker="x")
    for i in range(embs.shape[0]):
        ax.text(embs[i,0], embs[i,1], f"{i}", **style)
        ax.text(embs_dec[i,0], embs_dec[i,1], f"{i}", **style)
    plt.legend()

    plt.show()

In [ ]:
x_tilde, z_e_x, z_q_x = vqvae_toy.forward(torch.Tensor(X_test[:300]))

In [ ]:
vqvae_toy = VectorQuantizedVAE(latent_dim = 2, K=8, output_dim=2, beta=0.1, gumbel=True, hidden_dim=40, num_hidden=1, archi="basic", data_type="continuous")
train(vqvae_toy, 50)

In [ ]:
display_vq(vqvae_toy)

### VQ-VAE with Gumbel-Softmax on FashionMNIST

In [ ]:
train_loader, val_loader = make_dataloaders("fashionmnist", 32, device, subset=32*32, binarize=False, flatten=False)

In [ ]:
vqvae = VectorQuantizedVAE(latent_dim = 64, gumbel=True, tau=1., alpha=1., beta=1., K=128, num_hidden=0, output_dim=784, hidden_dim=128, normalize=True, activation="relu", archi="convMnist", data_type="continuous")

In [ ]:
train(vqvae, 200)

In [ ]:
latents = vqvae.encode(batch)
rec = vqvae.decode(latents)

In [ ]:
plt.figure(figsize=(16, 2))
for idx in range(16):
    plt.subplot(2, 16, idx+1)
    plt.axis('off');
    plt.imshow(batch[idx].data.view(28,28).numpy(), cmap="gray");
    plt.subplot(2, 16, idx+17)
    plt.axis('off');
    plt.imshow(rec[idx].data.view(28,28).numpy(), cmap="gray");

In [ ]:
latents = vqvae.encode(batch)
l = latents[7:8]
print(l[0])
plt.imshow(vqvae.decode(l)[0].data.view(28,28).numpy(), cmap="gray");